In [3]:
import pymysql

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

raspberryPiID = input("라즈베리파이의 ID를 입력: ")
raspberryPiPW = input('비밀번호를 입력: ')
name = input('냉장고 이름을 입력: ')
location = input('냉장고의 위치를 입력: ')
cameraID = input('카메라의 ID를 입력: ')

sql = "insert into refrigerator values(%s, %s, %s, %s)"
curs.execute(sql, (raspberryPiID, raspberryPiPW, name, location))
conn.commit()

sql = "insert into camera values(%s, %s)"
curs.execute(sql, (cameraID, raspberryPiID))
conn.commit()

with open(filename, mode = 'w') as f:
    f.write(raspberryPiID + ' ' + cameraID)
    
with open(filename, mode = 'r') as f:
    print(f.read())
conn.close()

라즈베리파이의 ID를 입력: 1
비밀번호를 입력: 1
냉장고 이름을 입력: 1
냉장고의 위치를 입력: 1
카메라의 ID를 입력: 1
(('1', '1', '1', '1'), ('applemango', 'applemango!1', '졸프용 냉장고', '한성대'), ('test1', 'test1!', '테스트용', '집'), ('test2', 'test2', '테스트', '집'), ('test3', 'test3', 'home', 'home'))
(('1', '1'), ('camera1', 'applemango'), ('camera2', 'test2'), ('camera3', 'test3'))
1 1


In [7]:
import pymysql

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

sql = "select photoID from photo where cameraID=%s"
curs.execute(sql, 'camera1')
rows = curs.fetchall()
pre = ''
if len(rows) != 0:
    pre = rows[0][0];       
    pre_split = pre.split('_')            
    for row in rows :    
        row_split = row[0].split('_')
        for i in range(0, len(pre_split)) :
            if(int(pre_split[i]) < int(row_split[i])) :
                pre = row[0]
                pre_split = pre.split('_')
                break;
print(pre)

2020_4_21_13_33_45


In [33]:
import pymysql
from datetime import datetime, timedelta

conn = pymysql.connect(host='13.125.102.154', user='hello', password='Csedbadmin!1', db='pytest')
curs = conn.cursor()

sql = "select photoID from photo where cameraID=%s"
curs.execute(sql, 'camera1')
rows = curs.fetchall()
print(rows)

now = '2020_4_21_14_6_32'
now_split = now.split('_')
now_time = datetime(int(now_split[0]), int(now_split[1]), int(now_split[2]), int(now_split[3]), int(now_split[4]), int(now_split[5]))
    
pre = ''
if len(rows) != 0:
    pre = rows[0][0];       
    pre_split = pre.split('_') 
    pre_time = datetime(int(pre_split[0]), int(pre_split[1]), int(pre_split[2]), int(pre_split[3]), int(pre_split[4]), int(pre_split[5]))
    for row in rows :    
        row_split = row[0].split('_')
        row_time = datetime(int(row_split[0]), int(row_split[1]), int(row_split[2]), int(row_split[3]), int(row_split[4]), int(row_split[5]))
        print(pre_time, row_time)
        if now_time == row_time:
            pass
        elif pre_time < row_time :
            pre = row[0]
            pre_split = pre.split('_')
            pre_time = datetime(int(pre_split[0]), int(pre_split[1]), int(pre_split[2]), int(pre_split[3]), int(pre_split[4]), int(pre_split[5]))
print(pre)
conn.close()

()

